### Code to copy mark points and fov pictures from raw to proc data for easier access
TODO: add this to the end of the suite2p preprocessing pipeline (e.g. check for each dataset if it exists)

In [ ]:
import os
import shutil

In [ ]:
# find any folder that contains 'fov' two levels down

def find_fov_folders(root):
    """
    Find all folders that contain 'fov' in their names, two levels down from the root directory.
    Also find all markpoint files in TSeries folders
    """
    fov_full_paths = []
    markpoint_full_paths = []

    all_mice = [d for d in os.listdir(root) if os.path.isdir(os.path.join(root, d)) and d.startswith('jm')]

    for mouse in all_mice:
        print(f"Finding fov and markpoints in mouse: {mouse}")
        mouse_path = os.path.join(root, mouse)

        mouse_sessions = [d for d in os.listdir(mouse_path) if d.startswith('20') and 'zstack' not in d]

        for session in mouse_sessions:

            session_path = os.path.join(mouse_path, session)
            # check if in session path there is a fov directory

            fov_path = os.path.join(session_path, 'fov')
            if os.path.isdir(fov_path):
                print(f"Found fov directory in {session_path}")
                fov_full_paths.append(fov_path)
            else:
                print(f"No 'fov' directory found in {session_path}, skipping")
                                      
            print(f"Checking session: {session}")
            print(f"Session lower: {session.lower()}")

            tseries_folders = [dir for dir in os.listdir(session_path) if 'TSeries' in dir]
            print(f"TSeries folders found: {tseries_folders}")

            if len(tseries_folders) == 0:
                print(f"No TSeries folders found in {session_path}, skipping")
                # go to next iteration of session loop
                continue


            elif len(tseries_folders) > 1:
                raise ValueError(f"Found more than one TSeries folder in {session_path}, found: {tseries_folders}")
            
            # tseries_folders should only have one element
            tseries_path = os.path.join(session_path, tseries_folders[0])

            markpoints_fname = [fname for fname in os.listdir(tseries_path) if 'MarkPoints.xml' in fname and not fname.startswith('.')]
            print(f"Markpoints found: {markpoints_fname}")

            if len(markpoints_fname) == 1:
                markpoints_path = os.path.join(tseries_path, markpoints_fname[0])
                markpoint_full_paths.append(markpoints_path)

            elif len(markpoints_fname) == 0:
                print(f"No MarkPoints.xml found in {tseries_path}, skipping")
            
            else:
                raise ValueError(f"Found more than one MarkPoints.xml file in {tseries_path}, found: {markpoints_fname}")
            
    print(f"Found {len(fov_full_paths)} fov folders")
    print(f"Found {len(markpoint_full_paths)} markpoint files")

    return fov_full_paths, markpoint_full_paths

def copy_folders_from_list(data_raw_root, data_proc_root, folder_list):
    """
    Copy folders from data_raw_root to data_proc_root based on a list of folder paths.
    """
    
    for folder in folder_list:
        # replace the data_raw_root with data_proc_root in the folder path
        new_folder = folder.replace(data_raw_root, data_proc_root)
        # create the new folder if it doesn't exist
        shutil.copytree(folder, new_folder, dirs_exist_ok=True)
        print(f"Copied {folder} to {new_folder}")

def copy_files_from_list(data_raw_root, data_proc_root, file_list):
    """
    Copy files from data_raw_root to data_proc_root based on a list of file paths.
    """
    
    for file in file_list:
        # replace the data_raw_root with data_proc_root in the file path
        new_file = file.replace(data_raw_root, data_proc_root)

        # remove the timeseries part of the path
        new_file = os.path.join(new_file.split("/T")[0], 'T' + new_file.split("/T")[1]+'.xml')

        # create the directory if it doesn't exist
        os.makedirs(os.path.dirname(new_file), exist_ok=True)
        shutil.copy(file, new_file)
        print(f"Copied {file} to {new_file}")

In [1]:
data_raw_root = 'data_raw/jm'
data_proc_root = 'data_proc/jm'

In [4]:
fov_full_paths, markpoint_full_paths = find_fov_folders(data_raw_root)


Finding fov and markpoints in mouse: jm046
No 'fov' directory found in data_raw/jm/jm046/2024-09-09_b, skipping
Checking session: 2024-09-09_b
Session lower: 2024-09-09_b
TSeries folders found: ['TSeries-10032023-1822-008']
Markpoints found: []
No MarkPoints.xml found in data_raw/jm/jm046/2024-09-09_b/TSeries-10032023-1822-008, skipping
No 'fov' directory found in data_raw/jm/jm046/2024-09-03_a, skipping
Checking session: 2024-09-03_a
Session lower: 2024-09-03_a
TSeries folders found: ['TSeries-10032023-1822-004']
Markpoints found: []
No MarkPoints.xml found in data_raw/jm/jm046/2024-09-03_a/TSeries-10032023-1822-004, skipping
No 'fov' directory found in data_raw/jm/jm046/2024-09-09_a, skipping
Checking session: 2024-09-09_a
Session lower: 2024-09-09_a
TSeries folders found: ['TSeries-10032023-1822-005']
Markpoints found: []
No MarkPoints.xml found in data_raw/jm/jm046/2024-09-09_a/TSeries-10032023-1822-005, skipping
No 'fov' directory found in data_raw/jm/jm046/2024-09-05_a, skipping


In [12]:
# copy_folders_from_list(data_raw_root, data_proc_root, fov_full_paths)
copy_files_from_list(data_raw_root, data_proc_root, markpoint_full_paths)

Copied data_raw/jm/jm052/2025-05-24_b/TSeries-10032023-1822-021/TSeries-10032023-1822-021_Cycle00001_MarkPoints.xml to data_proc/jm/jm052/2025-05-24_b/TSeries-10032023-1822-021.xml
Copied data_raw/jm/jm052/2025-05-23_b/TSeries-10032023-1822-021/TSeries-10032023-1822-021_Cycle00001_MarkPoints.xml to data_proc/jm/jm052/2025-05-23_b/TSeries-10032023-1822-021.xml
Copied data_raw/jm/jm048/2025-05-08_c/TSeries-10032023-1822-009/TSeries-10032023-1822-009_Cycle00001_MarkPoints.xml to data_proc/jm/jm048/2025-05-08_c/TSeries-10032023-1822-009.xml
Copied data_raw/jm/jm048/2025-05-06_b/TSeries-10032023-1822-008/TSeries-10032023-1822-008_Cycle00001_MarkPoints.xml to data_proc/jm/jm048/2025-05-06_b/TSeries-10032023-1822-008.xml
Copied data_raw/jm/jm048/2025-05-07_c/TSeries-10032023-1822-006/TSeries-10032023-1822-006_Cycle00001_MarkPoints.xml to data_proc/jm/jm048/2025-05-07_c/TSeries-10032023-1822-006.xml
Copied data_raw/jm/jm048/2025-05-12_c/TSeries-10032023-1822-006/TSeries-10032023-1822-006_Cycle